Перед запуском убедитесь, что в корне проекта есть файл .env и в нем заполнены выданные вам креды подключения к базам данных и хранилищу

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd
from dotenv import load_dotenv, find_dotenv
from sqlalchemy import create_engine, MetaData, Table


In [3]:
# подгружаем .env
load_dotenv()

True

In [4]:
# Считываем все креды
src_host = os.environ.get('DB_SOURCE_HOST')
src_port = os.environ.get('DB_SOURCE_PORT')
src_username = os.environ.get('DB_SOURCE_USER')
src_password = os.environ.get('DB_SOURCE_PASSWORD')
src_db = os.environ.get('DB_SOURCE_NAME') 

dst_host = os.environ.get('DB_DESTINATION_HOST')
dst_port = os.environ.get('DB_DESTINATION_PORT')
dst_username = os.environ.get('DB_DESTINATION_USER')
dst_password = os.environ.get('DB_DESTINATION_PASSWORD')
dst_db = os.environ.get('DB_DESTINATION_NAME')

s3_bucket = os.environ.get('S3_BUCKET_NAME')
s3_access_key = os.environ.get('AWS_ACCESS_KEY_ID')
s3_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')

In [5]:
# Создадим соединения
src_conn = create_engine(f'postgresql://{src_username}:{src_password}@{src_host}:{src_port}/{src_db}')
dst_conn = create_engine(f'postgresql://{dst_username}:{dst_password}@{dst_host}:{dst_port}/{dst_db}')

In [6]:
# Укажем таблицу для удаления
TABLE_NAME = 'users_churn'

In [7]:
# Функция удаления таблицы с помощью sqlalchemy
def delete_table(table_name, conn):
    from sqlalchemy import inspect
    inspector = inspect(conn)
    if table_name in inspector.get_table_names():
        metadata = MetaData(bind=conn)
        table = Table(table_name, metadata, autoload=True)
        table.drop(conn)
        print(f'Таблица {table_name} успешно удалена')
    else:
        print(f'Таблица {table_name} не существует в базе данных')

In [8]:
# Удалим таблицу
delete_table(TABLE_NAME, dst_conn)

Таблица users_churn не существует в базе данных


In [9]:
# Проверим, что таблицы больше нет
from sqlalchemy import inspect
inspector = inspect(dst_conn)
if TABLE_NAME in inspector.get_table_names():
    print(f'Таблица {TABLE_NAME} все еще существует')
    pd.read_sql(f'select * from {TABLE_NAME}', dst_conn)
else:
    print(f'Таблица {TABLE_NAME} успешно удалена - её больше нет в базе данных')

Таблица users_churn успешно удалена - её больше нет в базе данных


In [10]:
# Закроем соединения в конце работы
src_conn.dispose()
dst_conn.dispose()